# StockProject LASSO

- StockProject旨在使用股票市场过去的历史数据，对未来的股票收益率（return）进行尽可能精确的预测。  
- 本文档使用的是Lasso。

# Package

sklearn仅允许在cpu上训练，且该包相对古老，为了提升sklearn运行速度，我们采取如下办法：

In [1]:
#sklearn 加速
!pip install scikit-learn scikit-learn-intelex -i https://pypi.douban.com/simple/
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

In [8]:
#数据读取
import os
import pyarrow.feather as feather

#数据处理
import pandas as pd
import numpy as np

#进程展示
from tqdm import tqdm

#sklearn
from sklearn.linear_model import Lasso , LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

#超参调整
import optuna

#存贮模型
import pickle

#作图
import matplotlib.pyplot as plt 

# 数据预处理

 导入处理后的WRDS股票数据`/home/mw/input/stock3636/chars60_rank_imputed.feather`，并进行简单的数据预处理：  
 -  通过滞后一期，让当期的变量中包含需预测的变量（下一期的回报率）  
 -  删除分类变量（通过emedding和label encoder发现收效甚微）

In [2]:
#导入数据
with open('/home/mw/input/stock3636/chars60_rank_imputed.feather', 'rb') as f:
    data = feather.read_feather(f)
data['date'] = data['date'].astype('datetime64')


# 滞后代码
#我们的预测变量为下一期的股票收益率，需将下一期的股票收益率挪至本期
data['year_month'] = pd.to_datetime(data['date']).dt.to_period('M')
data['ret_fut'] = data.groupby('permno')['ret'].shift(-1)
data = data.dropna(axis=0,subset=['ret_fut']) #删除有空缺值的行

data.set_index('date', inplace=True)

# 缺失值处理
## 查看缺失值--没有缺失值
print('Missing data: {} items\n'.format(len(data[data.isna().any(1)])), data[data.isna().any(1)]) # 看一下缺失值是哪些行

#删除分类变量--embedding后约等于没有作用且速度慢
s = (data.dtypes == 'int64')
object_cols = list(s[s].index)# 移除含有类别变量的列
# 移除数据集含有类别变量的列
data = data.drop(object_cols, axis=1)

#删除影响数据分析的变量
data = data.drop(['rank_mom36m','rank_mom60m','exchcd','shrcd','lag_me','log_me'],axis=1)

# 模型 Lasso

我们定义了一个名为`Factor_models` 的类，该类旨在实现预测股票收益率并计使用样本外R方评估模型：  

$R_{OOS}^2 =  1 - \frac{\sum_{it} (ret_{it} -\hat{ret}^2_{it} )}{\sum_{it} ret_{it}^2}$  

这里$\hat{ret}_{it}$代表模型预测的第$i$只股票在$t$时期的收益率（return）  

- 在类的初始化方法 `__init__` 中，需要传入股票数据 `data`、初始训练时期长度 `train_period `和训练集扩展的频率 `freq`（默认为按月）。该方法用于对类的参数进行初始化。  

- `predict_ret` 方法用于预测股票收益率。其根据数据的时间索引确定训练和测试日期，并**逐月拆分训练集和测试集**。然后，对训练数据进行标准化处理，使用`Lasso`对训练集进行拟合，并预测测试集的收益率。预测结果和真实值被存储在一个数据帧中，并在每个训练结束日期打印样本外预测的R2指标。最后，将R2指标保存为CSV文件，并返回包含预测结果的数据帧。  

- `cal_oos` 方法用于计算样本外的R2指标。它首先检查是否已经运行过 `predict_ret` 方法，如果是，则直接使用已有的预测结果数据帧，否则先调用` predict_ret` 方法进行预测。然后，根据预测结果计算样本外的R2指标，并绘制不同模型的样本外R2柱状图。最后，返回包含不同模型样本外R2指标的数据系列。

In [10]:
class Factor_models(object):

    def __init__(self,data,train_period,freq='m'):
      
       #参数初始化
        self.data = data
        self.train_period = train_period #初始训练时期长度
        self.freq = freq                 #训练集expanding的频率，是按月还是按年还是其他
        
    def predict_ret(self):
        dates = self.data.index.unique()
        dates = dates.sort_values()
        test_dates= dates[self.train_period:len(dates)]

        
        # 创建一个train_end_list，训练集每月expanding。
        preddf = pd.DataFrame() # 存储不同模型预测出来的y值，即存储样本外预测收益率的值
        #记录R方
        R2df = pd.DataFrame() # 存储不同模型预测出来的y值，即存储样本外预测收益率的值


        
        for end_date in tqdm(test_dates,desc='Spilt and Train'): # 通过逐月改变训练集end_date的方法，切割样本
            
            #训练用数据
            train_temp = self.data[self.data.index <  end_date]
            test_temp = self.data[self.data.index == end_date]
            
            #测试集
            y_test = test_temp.ret_fut
            X_test = test_temp.drop(['ret_fut','ret','year_month'], axis=1)

            #预测训练数据集
            y_ptrain = train_temp.ret_fut
            X_ptrain = train_temp.drop(['ret_fut','ret','year_month'], axis=1)
            
            #对数据进行逐列标准化
            s = (X_ptrain.dtypes == 'float64')
            object_cols = list(s[s].index)

            scaler = StandardScaler()
            for col in object_cols:
                scaler.fit(X_test[col].values.reshape(-1,1))
                X_test[col] = scaler.transform(X_test[col].values.reshape(-1,1))
                scaler.fit(X_ptrain[col].values.reshape(-1,1))
                X_ptrain[col] = scaler.transform(X_ptrain[col].values.reshape(-1,1))
    
            # 建模预测收益率
            ## 先创建一个临时的temp_preddf,用来存储当前月份的验证集下的real y和不同模型的预测y
            temp_preddf = pd.DataFrame() # 创建当前训练集下训练出的predict y和real y
            y_test_rec = y_test.values.reshape(-1,1) #转为numpy
            temp_preddf['real_y'] = y_test_rec[:,0]# real_y就是验证集valid_y的第一列。因为valid_y是真实收益率数据在vailid_date上的切割                   
        
            #lasso
            lasso = Lasso() # 根据lasso模型预测收益率
            param = [{'alpha':np.linspace(0.0001,0.4,10)}]
            grid = GridSearchCV(lasso, param_grid=param) # 网格法调整超参
            grid.fit(X_ptrain,y_ptrain)
            y_predict = grid.predict(X_test)
                      
            ## 将temp_preddf并入preddf
            temp_preddf['Lasso_y'] = y_predict
            preddf = preddf.append(temp_preddf) # 将当前valid_date下得到的predict_y和real_y一起并入preddf中
            self._preddf = preddf
            
            #R2
            denominator = (preddf['real_y'] ** 2).sum() # 分母是真实收益率的平方和
            numerator = preddf.apply(lambda x: preddf['real_y'] - x).iloc[:,1:] # 分子是real_y - predict_y的平方和
            numerator = (numerator ** 2).sum()
            R2 = 1 - numerator / denominator # 再用 1 减去分子/分母
            print("==================")
            print("enddate:",end_date)
            print("Out-of-sample predicting R2:",R2)
            print("==================")
            R2df = R2df.append(R2,ignore_index=True )
            
            ## 将temp_preddf并入preddf
            preddf = preddf.append(temp_preddf) # 将当前valid_date下得到的predict_y和real_y一起并入preddf中
            self._preddf = preddf

        # 将数据帧保存为 CSV 文件
        R2df.to_csv('/home/mw/project/recording/R2_LASSO.csv', index=False)
        
        return preddf # 最后我们只返回preddf，也就是所有期的predict y和real y
    
    def cal_oos(self):
        # 计算out-of-sample R2 根据代码开头的公式
        try:
            preddf = self._preddf # 如果self已经有self._preddf，即self.predict_ret()已经运行过了，已经预测过收益率了，则无需再次运行。
        except:
            preddf = self.predict_ret() # 如果之前没有运行过self.predict_ret()，则需要运行。
        denominator = (preddf['real_y'] ** 2).sum() # 分母是真实收益率的平方和
        numerator = preddf.apply(lambda x: preddf['real_y'] - x).iloc[:,1:] # 分子是real_y - predict_y的平方和
        numerator = (numerator ** 2).sum()
        
        roos = 1 - numerator / denominator # 再用 1 减去分子/分母
        roos.index = roos.index.str.rstrip('_y') # 之前的index都是模型_y，比如"OLS_y"，不美观，删除_y。
        fig,ax = plt.subplots(figsize = (16,12)) # 画图，将不同模型的Roos画出来。
        plt.title('Out-of-sample predicting R2', fontsize = 20)
        ax.bar(x = roos.index, height = roos)
        plt.show()
        return roos # 返回样本外Roos，这个Roos是不同模型对应的样本外R2

# 预测

In [11]:
basic_2_factors = Factor_models(data,32,freq='m') #2013年只有8个月的数据

In [3]:
# 计算样本外R2，运行耗时较长
roos = basic_2_factors.cal_oos() # 计算不同模型样本外R2。self.cal_oos()中已经包含了self.predict_ret()的操作，先通过不同的模型预测收益率，再比较样本外真实收益率和预测收益率的差异
roos.to_csv('/home/mw/project/Lasso_tuning_roos.csv')